**Run All**: 

* If you are on SageMaker notebook instance *go to Cell tab -> Run All*
* If you are on SageMaker Studio *go to Run tab -> Run All Cells*

## Deep Demand Forecasting Demo

In this demo notebook, we can quickly send some data to an already deployed endpoint, get the response and visualize its results.

**To see more details of end-to-end model training and deployement using SageMaker, please checkout `deep-demand-forecast.ipynb` notebook.**

In [ ]:
import boto3
import botocore
import json
import numpy as np
from sagemaker.s3 import S3Downloader

Here we setup a SageMaker runtime and download some historical electricity consumption data for predictions. Data taken from *Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php), Irvine, CA: University of California, School of Information and Computer Science*.

In [ ]:
DATASET_NAME = 'electricity'
NUM_TS = 321
CONTEXT_LENGTH = 12
PREDICTION_LENGTH = 6

sagemaker_config = json.load(open('stack_outputs.json'))
config = botocore.config.Config(read_timeout=200)
runtime = boto3.client('runtime.sagemaker', config=config)

original_data_bucket = 'sagemaker-solutions-{}'.format(sagemaker_config['AWSRegion'])
original_data_prefix = 'sagemaker-deep-demand-forecast/data/{}'.format(DATASET_NAME)
original_data = 's3://{}/{}'.format(original_data_bucket, original_data_prefix)
print("original data: ")
S3Downloader.list(original_data)
!mkdir -p raw_data
!aws s3 cp --recursive $original_data raw_data

Now, we quickly process our `raw_data` in a pandas dataframe

In [ ]:
%run deep_demand_forecast/monitor.py

In [ ]:
train_df, test_df = prepare_data('raw_data')
print(f'raw train data shape {train_df.shape}, test data shape {test_df.shape}')
ts_col_names = [f'ts_{i}' for i in range(NUM_TS + 1)]
train_df_viz, test_df_viz, selected_cols = create_data_viz(train_df, test_df, CONTEXT_LENGTH, PREDICTION_LENGTH, num_sample_ts=11)
train_df_viz.head()

And we prepare our test data as json. Then, invoke our endpoint for getting some predictions

In [ ]:
%%time
num_train = train_df.shape[0] - 1
test_data = {}
test_data['target'] = test_df.iloc[num_train: num_train + PREDICTION_LENGTH].set_index('time').values.T.tolist()
test_data['start'] =  '2014-05-26 19:00:00'
test_data['source'] = []

payload = json.dumps(test_data)
response = runtime.invoke_endpoint(EndpointName=sagemaker_config['SolutionPrefix'] + '-demo-endpoint',
                                   ContentType='application/json',
                                   Body=payload)
predictions = json.loads(response['Body'].read().decode())

Finally, we prepare the predictions for interactive visualization using [GluonTS API](https://ts.gluon.ai/) and [Altair](https://altair-viz.github.io/)

In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

forecasts = np.transpose(np.array(predictions['forecasts']['samples'][0]))
preds = ListDataset([{FieldName.TARGET: forecasts,
                           FieldName.START: predictions['forecasts']['start_date']
                      }], freq=predictions['forecasts']['freq'], one_dim_target=False)

preds_df = multivar_df(next(iter(preds)))
preds_df_filter = preds_df.loc[:, ['time'] + selected_cols]
preds_df_filter = pd.melt(preds_df_filter, id_vars=['time'], value_vars=selected_cols)
preds_df_filter.rename(columns={'variable': 'covariate'}, inplace=True)
preds_df_filter.head()

In [ ]:
import altair as alt

selection = alt.selection_multi(fields=['covariate'], bind='legend', nearest=True)
preds_plot = alt.Chart(preds_df_filter, title='Predictions').mark_line().encode(
    alt.X('time:T', axis=alt.Axis(title='Time')),
    alt.Y('value:Q', axis=alt.Axis(title='Normalized electricity consumption (kW)')),
    alt.Color('covariate:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)

preds_plot

**To see more details of end-to-end model training and deployement using SageMaker, please checkout `deep-demand-forecast.ipynb` notebook.**